<a href="https://colab.research.google.com/github/kr-pushpam/DeepGenerativeModel_Aug24/blob/main/CNN_Classification_Animal_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import gdown  # gdown is used to download files from Google Drive
import zipfile  # zipfile is used to handle zip files (like extracting them)
import os

# Google Drive file ID and link for the dataset (source: Google Drive)
# The file_id corresponds to the file you want to download from Google Drive
# Construct the direct download URL using the file_id, link that allows direct download

file_id = '1lBkM874rRc8aTQAI3gn67urH9jh1GgmJ'
download_url = f"https://drive.google.com/uc?id={file_id}"

output = '/content/animal_dataset.zip'# Define the destination path for the downloaded zip file in Colab's local filesystem
gdown.download(download_url, output, quiet=False) # Download the zip file from Google Drive to Colab's local storage i.e. output path

extracted_path = '/content/'# Define the extraction path in Colab's local storage where the dataset will be extracted, root in this case

# Unzip the downloaded dataset from the zip file (source: the downloaded zip file)
# Unzipping the zip file located at 'output' and extracting it to 'extracted_path' , root in this case
with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall(extracted_path)  # Extract all files to /content/

# This lists all the files and directories in the /content/ directory (where the dataset was extracted)
print("Extracted File", os.listdir(extracted_path))

Downloading...
From (original): https://drive.google.com/uc?id=1lBkM874rRc8aTQAI3gn67urH9jh1GgmJ
From (redirected): https://drive.google.com/uc?id=1lBkM874rRc8aTQAI3gn67urH9jh1GgmJ&confirm=t&uuid=5fed892e-ac19-4c69-a238-1730baa93fc2
To: /content/animal_dataset.zip
100%|██████████| 701M/701M [00:02<00:00, 305MB/s]


Extracted File ['.config', 'animal_dataset.zip', 'drive', 'Animals_data', 'sample_data']


In [4]:
# file management utilities

import shutil
import os
dir_path = "/content/content/Animals_data_backup"

# shutil.rmtree(dir_path) # remove or delete a given directory
shutil.copytree("Animals_data", "Animals_data_backup") # copy a given directory

'Animals_data_backup'

In [10]:
import torch
import torch.nn as nn
import os
import time
from tqdm import tqdm
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
from datetime import date
from torchvision import models
from PIL import Image
import torch.nn.functional as F
import torch.optim as optim

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
if not os.path.exists('/content/working/weights/'):
    os.makedirs('/content/working/weights/')


cuda


In [7]:
dataset_dir = "/content/Animals_data_backup/animals/animals/"
bsz = 16

data_transforms = transforms.Compose([
    transforms.Resize((240, 240)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                        [0.229, 0.224, 0.225])
])

#split_data
train_dataset = datasets.ImageFolder(root=dataset_dir, transform=data_transforms)
train_data, val_data = torch.utils.data.random_split(train_dataset, [int(train_dataset.__len__()*0.8), train_dataset.__len__()-int(train_dataset.__len__()*0.8)])
#load_data
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=bsz, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=bsz, shuffle=False)

In [8]:
print(f"Number of training samples: {len(train_data)}")
print(f"Number of validation samples: {len(val_data)}")

# Example: Iterate over one batch
train_features, train_labels = next(iter(train_dataloader))

print("Train features batch shape:", train_features.shape)
print("Train labels batch shape:", train_labels.shape)

Number of training samples: 4320
Number of validation samples: 1080
Train features batch shape: torch.Size([16, 3, 240, 240])
Train labels batch shape: torch.Size([16])


In [11]:
# Define the CNN model (model 3)
class Model3(nn.Module):
    def __init__(self, num_classes=90):
        super(Model3, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)  # Conv2D with padding='same'
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # MaxPool2D(pool_size=(2, 2), strides=2)
        self.flatten = nn.Flatten()

        # Instead of hardcoding, use AdaptiveAvgPool2d to ensure the output is fixed-size
        self.adaptive_pool = nn.AdaptiveAvgPool2d((7, 7))  # You can adjust this to match your final output size

        # Define the fully connected layer
        self.fc = nn.Linear(64 * 7 * 7, num_classes)  # 64 filters * 7x7 feature map

    def forward(self, x):
        x = F.relu(self.conv1(x))  # Activation
        x = self.pool(x)  # Pooling
        x = F.relu(self.conv2(x))  # Activation
        x = self.pool(x)  # Pooling

        # Apply adaptive pooling to get fixed size
        x = self.adaptive_pool(x)

        x = self.flatten(x)  # Flatten the feature maps into a single vector
        x = self.fc(x)  # Fully connected layer
        return F.log_softmax(x, dim=1)  # Log softmax for multi-class classification

# Instantiate model 3
model3 = Model3(num_classes=90)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for multi-class classification
optimizer = optim.Adam(model3.parameters(), lr=0.001)  # Adam optimizer with learning rate 0.001


In [13]:
# Check if a GPU is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Move the model to the device (GPU or CPU)
model3.to(device)

# Training the model3
num_epochs = 10  # Number of epochs
train_losses = []
val_losses = []

# Training and evaluation loop remains the same
for epoch in range(num_epochs):
    model3.train()  # Set model to training mode
    running_loss = 0.0

    for images, labels in train_dataloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        # Forward pass with model 3
        outputs = model3(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Validation step for model 3
    model3.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model3(images)
            val_loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {running_loss/len(train_dataloader):.4f}, Validation Accuracy: {100*correct/total:.2f}%')



Epoch [1/10], Training Loss: 4.2996, Validation Accuracy: 9.91%
Epoch [2/10], Training Loss: 3.6540, Validation Accuracy: 15.37%
Epoch [3/10], Training Loss: 3.1468, Validation Accuracy: 18.52%
Epoch [4/10], Training Loss: 2.7756, Validation Accuracy: 20.56%
Epoch [5/10], Training Loss: 2.4737, Validation Accuracy: 24.17%
Epoch [6/10], Training Loss: 2.1649, Validation Accuracy: 25.09%
Epoch [7/10], Training Loss: 1.8856, Validation Accuracy: 28.24%
Epoch [8/10], Training Loss: 1.6429, Validation Accuracy: 28.24%
Epoch [9/10], Training Loss: 1.4478, Validation Accuracy: 31.02%
Epoch [10/10], Training Loss: 1.2396, Validation Accuracy: 32.78%


In [14]:
# Evaluation on the validation data (testing phase)
model3.eval()  # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():  # Disable gradient calculations for faster evaluation
    for images, labels in val_dataloader:  # Use the validation dataloader for evaluation
        images, labels = images.to(device), labels.to(device)
        outputs = model3(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

val_accuracy = 100 * correct / total
print(f'Validation/Test Accuracy: {val_accuracy:.2f}%')

Validation/Test Accuracy: 32.78%


In [ ]:
# Save the model parameters to Google Drive
model_save_path = '/content/drive/MyDrive/Colab Notebooks/DGM -Aug-2024/cnn_classification_animal.pth'
torch.save(model3.state_dict(), model_save_path)

In [20]:
torch.save(model3.state_dict(), 'cnn_classification_animal.pth')  # Save the model parameters

In [25]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

# Save the model parameters to Google Drive
model_save_path = '/content/gdrive/MyDrive/Colab Notebooks/DGM -Aug-2024/cnn_classification_animal.pth'
torch.save(model3.state_dict(), model_save_path)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [24]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

# Define the path where the model will be saved/loaded
model_save_path = '/content/cnn_classification_animal.pth'

# Check if the model file already exists
if os.path.exists(model_save_path):
    print("Saved model found. Loading the model for fine-tuning...")

    # Recreate the model architecture
    model3 = Model3(num_classes=90)

    num_fine_tune_epochs = 10

    # Load the saved model weights
    model3.load_state_dict(torch.load(model_save_path))

    # Move model to the appropriate device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model3.to(device)

    # Set the model to training mode for fine-tuning
    model3.train()

    # Define optimizer and criterion for fine-tuning
    optimizer = torch.optim.Adam(model3.parameters(), lr=0.0001)  # Reduced learning rate for fine-tuning
    criterion = torch.nn.CrossEntropyLoss()

    # Continue training (fine-tuning)
    for epoch in range(num_fine_tune_epochs):  # Adjust fine-tuning epochs
        for images, labels in train_dataloader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()  # Zero the gradients
            outputs = model3(images)  # Forward pass
            loss = criterion(outputs, labels)  # Compute the loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights

        print(f'Epoch [{epoch+1}/{num_fine_tune_epochs}], Fine-tuning Loss: {loss.item():.4f}')

    # Save the fine-tuned model to Google Drive
    torch.save(model3.state_dict(), model_save_path)
    print("Fine-tuned model saved to Google Drive.")

else:
    print("No saved model found. Starting training from scratch...")

    # Create and initialize the model
    model3 = Model3(num_classes=90)

    # Move model to the appropriate device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model3.to(device)

    # Define optimizer and criterion for training from scratch
    optimizer = torch.optim.Adam(model3.parameters(), lr=0.001)  # Standard learning rate for initial training
    criterion = torch.nn.CrossEntropyLoss()

    # Train the model from scratch
    num_epochs = 10  # Adjust based on your needs
    for epoch in range(num_epochs):
        model3.train()
        running_loss = 0.0

        for images, labels in train_dataloader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()  # Zero the gradients
            outputs = model3(images)  # Forward pass
            loss = criterion(outputs, labels)  # Compute the loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights

            running_loss += loss.item()

        print(f'Epoch [{epoch+1}/{num_epochs}], Training Loss: {running_loss/len(train_dataloader):.4f}')

    # Save the model after initial training
    torch.save(model3.state_dict(), model_save_path)
    print("Model trained and saved to Google Drive.")



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Saved model found. Loading the model for fine-tuning...


<ipython-input-24-b91d45fb8148>:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model3.load_state_dict(torch.load(model_save_path))


Epoch [1/10], Fine-tuning Loss: 2.9037
Epoch [2/10], Fine-tuning Loss: 3.0176
Epoch [3/10], Fine-tuning Loss: 3.0339
Epoch [4/10], Fine-tuning Loss: 3.3918
Epoch [5/10], Fine-tuning Loss: 2.3815
Epoch [6/10], Fine-tuning Loss: 1.8801
Epoch [7/10], Fine-tuning Loss: 3.3500
Epoch [8/10], Fine-tuning Loss: 2.2007
Epoch [9/10], Fine-tuning Loss: 1.5112
Epoch [10/10], Fine-tuning Loss: 2.0468
Fine-tuned model saved to Google Drive.
